<a href="https://colab.research.google.com/github/Vsingh31/NewProject/blob/main/spark_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [4]:
# Order a DataFrame by ascending values
df.orderBy(df['points'].asc()).show()

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    country|         description|         designation|              points|               price|            province|            region_1|            region_2|             variety|              winery|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Italy|This offers gener...|                null|                null|                null|                null|                null|                null|                null|                null|
|         US|               Ripe |                null|                null|                null|                null|                null|                null|                null|       

In [6]:
# Order a DataFrame by descending values
df.orderBy(df['points'].desc()).show()

+-------+--------------------+--------------------+------+-----+------------+--------------------+---------------+--------------------+--------------------+
|country|         description|         designation|points|price|    province|            region_1|       region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+------------+--------------------+---------------+--------------------+--------------------+
|     US|The only one of t...|   Cailloux Vineyard|    99|   65|      Oregon|Walla Walla Valle...|   Oregon Other|               Syrah|              Cayuse|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385|   Champagne|           Champagne|           null|     Champagne Blend|     Mo��t & Chandon|
|     US|Depth and texture...|Royal City Stoner...|    99|  140|  Washington|Columbia Valley (WA)|Columbia Valley|               Syrah|       Charles Smith|
| France|98-100 Barrel sam...|       Barrel sample|    99|

In [7]:
# Import average function
from pyspark.sql.functions import avg
df.select(avg('points')).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [10]:
# Using filter
df.filter('price>20').show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [11]:
# Filter by price on certain columns
df.select('price','winery','points','country').filter('price>100').show()

+-----+--------------------+------+--------+
|price|              winery|points| country|
+-----+--------------------+------+--------+
|  235|               Heitz|    96|      US|
|  110|Bodega Carmen Rod...|    96|   Spain|
|  110|Bodega Carmen Rod...|    95|   Spain|
|  185|      Domaine Serene|    95|      US|
|  325|                Hall|    95|      US|
|  290|  Ch̢teau Lagr̩zette|    95|  France|
|  220|           Numanthia|    95|   Spain|
|  105|               Ponzi|    94|      US|
|  135|            Carillon|    90|   Italy|
|  125|        Charles Krug|    86|      US|
|  120|Ch̢teau Raymond-L...|    93|  France|
|  200|       Diamond Creek|    93|      US|
|  155|            Parallel|    94|      US|
|  130|    Herdade do Rocim|    91|Portugal|
|  115|              Oddero|    91|   Italy|
|  125|       Sequoia Grove|    92|      US|
|  125|            21 Grams|    93|      US|
|  120|             Sbragia|    93|      US|
|  111|       Eleven Eleven|    90|      US|
|  110|  T

In [13]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|          region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|              Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|    Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|            Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
| France|This is the top w...|    

In [18]:
df.filter((df['price'] > 200) | (df['points'] < 80)).show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+--------------------+--------------------+
|       US|This tremendous 1...|   Martha's Vineyard|    96|  235|      California|         Napa Valley|    Napa|  Cabernet Sauvignon|               Heitz|
|       US|This blockbuster,...|     Rainin Vineyard|    95|  325|      California|Diamond Mountain ...|    Napa|  Cabernet Sauvignon|                Hall|
|   France|Coming from a sev...|       Le Pigeonnier|    95|  290|Southwest France|              Cahors|    null|              Malbec|  Ch̢teau Lagr̩zette|
|    Spain|Tarry blackberry ...|          Termanthia|    95|  22

In [19]:
df.filter(df["country"] == "US").show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Or